# Brownian Motion - Frequency of Crossing Origin

In [2]:
import random
import numpy as np
def position_after_t_steps(time_ticks: int, num_exps: int):
    if time_ticks <= 0 or num_exps <= 0:
        return
    observed_hops = np.zeros(num_exps).astype(int)
    for i in range(num_exps):
        # initial position of the particle
        particle_position = 0
        step_choices = 10
        # Generate an array having an equal number of +1 and -1 elements. The elements are shuffled
        # The particle will choose a step from this array every time and move
        steps = np.concatenate((np.ones(step_choices).astype(int), np.full(step_choices, -1).astype(int)))
        np.random.shuffle(steps)
        # tracking frequency of crossing the origin
        hops_across_origin = 0
        for time in range(time_ticks):
            # If the particle is currently at origin, it will now cross it
            if particle_position == 0:
                hops_across_origin += 1
            # Randomly choose a step
            particle_position += steps[random.randint(0,2 * step_choices - 1)]
        # Discounting the first "hop," when the particle is at origin initially
        observed_hops[i] = hops_across_origin - 1
    avg_hops = np.mean(observed_hops)
    print("t: %8d | No. of experiments: %3d | Average hops across origin: %4d" %(time_ticks, num_exps, avg_hops))

position_after_t_steps(40000, 50)
position_after_t_steps(90000, 50)
position_after_t_steps(160000, 50)
        
        

t:    40000 | No. of experiments:  50 | Average hops across origin:  154
t:    90000 | No. of experiments:  50 | Average hops across origin:  193
t:   160000 | No. of experiments:  50 | Average hops across origin:  274


# Trade-offs in sampling

In [1]:
import numpy as np
import numpy.typing as npt

# Setting things up
percentage_plus_votes = 0.52
percentage_minus_votes = 1 - percentage_plus_votes
vote_bank_size = 1000000
num_exps = 100

# Generate arrays of +1 and -1 votes, having a 52-48 split by size
plus_votes = np.ones(int(vote_bank_size * percentage_plus_votes)).astype(int)
minus_votes = np.full(int(vote_bank_size * percentage_minus_votes), -1).astype(int)

# Generate a single "vote bank" and shuffle it
all_votes = np.concatenate((plus_votes, minus_votes))
np.random.shuffle(all_votes) # in-place shuffling

def perform_experiment(population: npt.NDArray[np.int_], sample_size: int, num_exps: int):
    if num_exps <= 0 or sample_size <=0:
        return
    samples_with_majority = 0
    for exp in range(num_exps):
        # Sample votes without replacement
        sample = np.random.choice(a=population, size=sample_size, replace=False)
        # Keep track of the experiments which resulted in +1 being the majority
        if np.sum(sample) > 0:
            samples_with_majority += 1
    return samples_with_majority/num_exps


print(f'No. of plus votes: {len(plus_votes)}')
print(f'No. of minus     : {len(minus_votes)}')
print(f'Total votes      : {len(all_votes)}\n')

print('Performing experiment...\n')
sample_sizes = [20, 100, 400]
for sample_size in sample_sizes:
    prob_experiment = perform_experiment(all_votes, sample_size, num_exps)
    print(f"Sample size: %5s | Number of experiments: %3s | Probability that majority element is +1: %4.2f" % (sample_size, num_exps, prob_experiment))

print('Determining sample size such that the probability of getting the majority element is 0.9')
# Empirical results show that the sample size should be greater than 400 for probability > 0.9

exp_sample_size = 400
sample_step = 50
while exp_sample_size > 0:
    prob_experiment = perform_experiment(all_votes, exp_sample_size, num_exps)
    print(f"Sample size: %5s | Number of experiments: %3s | Probability that majority element is +1: %4.2f" % (exp_sample_size, num_exps, prob_experiment))
    if abs(0.9 - prob_experiment) <= 0.01:
        break
    if 0.9 > prob_experiment:
        sample_step = sample_step + int(sample_step / 2)
        exp_sample_size += sample_step
    else:
        sample_step = sample_step - int(sample_step / 2)
        exp_sample_size -= sample_step
    exp_sample_size += sample_step
                        



No. of plus votes: 520000
No. of minus     : 480000
Total votes      : 1000000

Performing experiment...

Sample size:    20 | Number of experiments: 100 | Probability that majority element is +1: 0.49
Sample size:   100 | Number of experiments: 100 | Probability that majority element is +1: 0.69
Sample size:   400 | Number of experiments: 100 | Probability that majority element is +1: 0.79
Determining sample size such that the probability of getting the majority element is 0.9
Sample size:   400 | Number of experiments: 100 | Probability that majority element is +1: 0.79
Sample size:   550 | Number of experiments: 100 | Probability that majority element is +1: 0.81
Sample size:   774 | Number of experiments: 100 | Probability that majority element is +1: 0.88
Sample size:  1110 | Number of experiments: 100 | Probability that majority element is +1: 0.91
Sample size:  1110 | Number of experiments: 100 | Probability that majority element is +1: 0.92
Sample size:  1110 | Number of experi